<a href="https://colab.research.google.com/github/s00hyun/Today-I-Learned/blob/master/ML-DL/deep-learning-pytorch/lec5_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
#!pip install import_ipynb

In [0]:
import import_ipynb

In [4]:
!pwd

/content


In [5]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/딥러닝수업

/content/gdrive/My Drive/Colab Notebooks/딥러닝수업


In [6]:
!pwd

/content/gdrive/My Drive/Colab Notebooks/딥러닝수업


In [7]:
!ls -l lec5_model.ipynb

-rw------- 1 root root 5687 Apr 14 06:35 lec5_model.ipynb


In [0]:
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as transforms

In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("current device:", device)

current device: cuda:0


In [0]:
num_epochs, num_classes, batch_size, learning_rate = 10, 10, 16, 0.01

In [0]:
from lec5_model import ConvNet, AlexNet, VGGNet

In [0]:
network = 'alexnet'  # simple-cnn, alexnet, vgg-16

In [0]:
if network == 'simple-cnn':
  model = ConvNet().to(device)
  composed_transforms = transforms.Compose([transforms.Resize((32, 32)),  # 해당 네트워크에 맞게 input size 조정
                                            # 텐서 Type으로 변환
                                            # Type: PIL -> torch.Tensor
                                            # Pixel value: [0, 255] -> [0, 1]
                                            transforms.ToTensor(),   
                                            # (R채널 Mean, G채널 Mean, B채널 Mean), (R채널 Std, G채널 Std, B채널 Std)
                                            # Pixel value를 [0, 1]에서 [-1, 1]로 변환함
                                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
elif network == 'alexnet':
  model = AlexNet(num_classes=num_classes).to(device)
  composed_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
elif network == 'vgg-16':
  model = VGGNet(num_classes=num_classes).to(device)
  composed_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [14]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=4096, out_features=4096, bias=True)

In [15]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=composed_transforms, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=composed_transforms)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

Files already downloaded and verified


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [17]:
# 모델에 학습 데이터를 입력 -> 예측 레이블 산출, 정답 레이블과 비교 -> 모델의 파라미터 값 업데이트

for epoch in range(num_epochs):
  # 각 data batch에 대해 학습
  # (image, labels): (입력 이미지, 정답 레이블)
  for i, (images, labels) in enumerate(train_loader):
    # to(device): 연산 수행 디바이스 설정
    # images, labels, model간 연산을 위해.
    images = images.to(device)  
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()

    if (i + 1) % 1000 == 0:
      print('Epoch: {}/{}, Batch Step: {}/{}, Loss: {:.4f}, Training Accuracy of the Current Batch: {}%'.
            format(epoch + 1, num_epochs, i + 1, train_loader.__len__(), loss.item(), 100 * correct / batch_size))


Epoch: 1/10, Batch Step: 1000/3125, Loss: 1.6694, Training Accuracy of the Current Batch: 56.25%
Epoch: 1/10, Batch Step: 2000/3125, Loss: 1.4801, Training Accuracy of the Current Batch: 50.0%
Epoch: 1/10, Batch Step: 3000/3125, Loss: 1.2393, Training Accuracy of the Current Batch: 56.25%
Epoch: 2/10, Batch Step: 1000/3125, Loss: 1.4334, Training Accuracy of the Current Batch: 43.75%
Epoch: 2/10, Batch Step: 2000/3125, Loss: 0.8669, Training Accuracy of the Current Batch: 81.25%
Epoch: 2/10, Batch Step: 3000/3125, Loss: 1.0446, Training Accuracy of the Current Batch: 56.25%
Epoch: 3/10, Batch Step: 1000/3125, Loss: 1.0018, Training Accuracy of the Current Batch: 68.75%
Epoch: 3/10, Batch Step: 2000/3125, Loss: 1.2166, Training Accuracy of the Current Batch: 56.25%
Epoch: 3/10, Batch Step: 3000/3125, Loss: 0.5934, Training Accuracy of the Current Batch: 75.0%
Epoch: 4/10, Batch Step: 1000/3125, Loss: 0.4379, Training Accuracy of the Current Batch: 81.25%
Epoch: 4/10, Batch Step: 2000/31

In [18]:
model.eval()  # 모델 평가 모드로 변경 (모델 변수 고정 등)
with torch.no_grad():  # 테스트 과정에서 gradient 계산 배제
  total, correct = 0, 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Test Accuracy of the 10,000 Test Images: {}%'.format(100 * correct / total))

Test Accuracy of the 10,000 Test Images: 68.24%
